In [8]:
#3.4.1 Con respecto a la librería multiprocessing, utilizar la función starmap para crear los procesos que se van a ejecutar en paralelo.
import numpy as np
from multiprocessing import Pool
import time 

# Modificamos la función de reducción para aceptar un inicio y fin
def sum_multiprocessing(A, ini, fin):
    return np.sum(A[ini:fin])

# Secuencial
value = 5 * 10** 7  # Tamaño del array
X = np.random.rand(value)  # Array con valores aleatorios
print(f"Tamaño del array: {X.size}\n")

# Tiempo secuencial
print("Tiempo para la operación de reducción secuencial:")
tiempo = %timeit -r 2 -o -q np.sum(X)
print(f"Medicion de tiempo usando numpy.sum(): {tiempo}")
print(f"Resultado de la suma secuencial: {np.sum(X)}\n")


# Divisiones del array para multiprocessing
def dividir_array(value, num_procesos):
    if num_procesos == 1:
        return [(0, value)]  # Una sola partición [0, value]
    elif num_procesos == 2:
        mitad = int(value / 2)
        return [(0, mitad), (mitad, value)]  # Dos particiones [0, mitad] y [mitad, value]
    elif num_procesos == 4:
        cuarto = int(value / 4)
        mitad = int(value / 2)
        tres_cuartos = int(3 * value / 4)
        return [(0, cuarto), (cuarto, mitad), (mitad, tres_cuartos), (tres_cuartos, value)]  # Cuatro particiones
    else:
        raise ValueError("Número de procesos no soportado. Use 1, 2 o 4.")

# Utilizando multiprocessing con 1, 2 y 4 procesos
for num_procesos in [1, 2, 4]:
    print(f"\nTiempo para la operación de reducción con {num_procesos} procesos:")

    # Dividimos el array según el número de procesos
    rangos = dividir_array(value, num_procesos)

    # Definimos la función que ejecutará multiprocessing
    def run_parallel_reduction():
        with Pool(processes=num_procesos) as pool:
            # Usamos starmap para aplicar sum_multiprocessing a cada rango
            resultados = pool.starmap(sum_multiprocessing, [(X, ini, fin) for (ini, fin) in rangos])
        return sum(resultados)

    # Medimos el tiempo usando %timeit
    tiempo = %timeit -r 2 -o -q run_parallel_reduction()
    resultado = run_parallel_reduction()

    print(f"Medicion de tiempo usando {num_procesos} processes: {tiempo}")
    print(f"Resultado de la suma con {num_procesos} procesos: {resultado}")

Tamaño del array: 50000000

Tiempo para la operación de reducción secuencial:
Medicion de tiempo usando numpy.sum(): 19.2 ms ± 8.13 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Resultado de la suma secuencial: 25002132.539449412


Tiempo para la operación de reducción con 1 procesos:
Medicion de tiempo usando 1 processes: 353 ms ± 2.59 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 1 procesos: 25002132.539449412

Tiempo para la operación de reducción con 2 procesos:
Medicion de tiempo usando 2 processes: 675 ms ± 5.38 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 2 procesos: 25002132.53944942

Tiempo para la operación de reducción con 4 procesos:
Medicion de tiempo usando 4 processes: 1.32 s ± 11.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 4 procesos: 25002132.539449368


In [9]:
import cupy as cp
import numpy as np
import time

# Crear el array en la CPU usando numpy
value = 5 * 10**7  # Tamaño del array
X_cpu = np.random.rand(value).astype(np.float32)  # Array en la CPU con valores aleatorios
print(f"Tamaño del array: {X_cpu.size}\n")

# Medir el tiempo de la transferencia de datos a la GPU
start_transfer_time = time.time()

# Copiar el array a la GPU usando cupy
X_gpu = cp.asarray(X_cpu)

end_transfer_time = time.time()
transfer_time = (end_transfer_time - start_transfer_time) * 1e6  # Convertir a microsegundos

# Medir el tiempo de la operación de reducción en la GPU
start_time = time.time()

# Realizar la suma en la GPU usando cupy
result_cupy_sum = cp.sum(X_gpu)

end_time = time.time()

# Medir el tiempo de ejecución de la suma en la GPU
time_taken = (end_time - start_time) * 1e6  # Convertir a microsegundos

# Copiar el resultado de vuelta a la CPU
result_cpu = cp.asnumpy(result_cupy_sum)

# Imprimir los resultados
print(f"Tiempo de transferencia a la GPU: {transfer_time:.2f} µs")
print(f"Tiempo usando cupy.sum(): {time_taken:.2f} µs")
print(f"Resultado de la suma usando cupy.sum(): {result_cupy_sum}\n")

# Verificar que el resultado es el mismo que en la CPU
print(f"Resultado en la CPU después de transferir desde la GPU: {result_cpu}")


Tamaño del array: 50000000

Tiempo de transferencia a la GPU: 41026.35 µs
Tiempo usando cupy.sum(): 134.71 µs
Resultado de la suma usando cupy.sum(): 24997542.0

Resultado en la CPU después de transferir desde la GPU: 24997542.0


In [12]:
import numba
import numpy as np
import time

# Crear el array en la CPU usando numpy
value = 5 * 10**7  # Tamaño del array
X_cpu = np.random.rand(value)  # Array en la CPU con valores aleatorios
print(f"Tamaño del array: {X_cpu.size}\n")
# Definir una función de reducción paralela usando numba.njit
@numba.njit(parallel=True)
def parallel_reduce(array):
    total = 0.0
    for i in numba.prange(array.size):  # Paralelizar el bucle con prange
        total += array[i]
    return total

# Medir el tiempo de ejecución de la reducción en la CPU con numba
start_time = time.time()

# Realizar la reducción
result_numba_sum = parallel_reduce(X_cpu)

end_time = time.time()

# Medir el tiempo de ejecución
time_taken = end_time - start_time

print(f"Medicion de tiempo usando  Numba parallel_reduce: {time_taken:.6f} seconds")
print(f"Resultado de la suma usando Numba parallel_reduce: {result_numba_sum}\n")

Tamaño del array: 50000000

Medicion de tiempo usando  Numba parallel_reduce: 0.240501 seconds
Resultado de la suma usando Numba parallel_reduce: 24999669.644815106



RESULTADO: los resultados obtenidos indican que para maximizar la velocidad en CPU, numpy.sum() sigue siendo la opción más rápida; cuando disponemos de la GPU, CuPy demostró ser el más eficiente; pero si no se tiene acceso a una GPU Numba parallel_reduce, ofrece una mejora significativa frente a las versiones multiproceso (que empeoran a medida que se agregan procesos).